In [ ]:
# Start Redis container
!docker run -d --name workshop-redis-module10 -p 6379:6379 redis:7-alpine

# Wait for Redis to be ready
import time
time.sleep(5)

# Test connection
!docker exec workshop-redis-module10 redis-cli ping

print('✅ Redis container is running on localhost:6379')

# Module 10: Troubleshooting & Migration

## 🎯 Interactive Lab: Diagnostics & Data Migration

**Duration:** 60 minutes  
**Level:** Advanced  

In this lab, you'll:
- 🔍 Use diagnostic commands to troubleshoot issues
- 📊 Analyze slow queries and performance
- 🚀 Migrate data between Redis instances
- 🛠️ Use RIOT for bulk operations
- ✅ Implement migration best practices

---


## 🐳 Start Docker Redis Container

Before we begin, let's start a Redis container using Docker:

## Part 1: Setup


In [ ]:
!pip install -q redis

import redis
import time
import json
from datetime import datetime

# Connect to Redis
r = redis.Redis(host='localhost', port=6379, decode_responses=True)

# Test connection
try:
    r.ping()
    print('✅ Connected to Redis')
except Exception as e:
    print(f'❌ Connection failed: {e}')


---

## Part 2: Diagnostic Commands

Essential commands for troubleshooting Redis:


In [ ]:
def run_diagnostics():
    """Run comprehensive Redis diagnostics"""
    
    print('🔍 Redis Diagnostics Report')
    print('=' * 60)
    
    # Server info
    info = r.info('server')
    print(f'\n📊 Server Info:')
    print(f'   Redis version: {info["redis_version"]}')
    print(f'   Uptime: {info["uptime_in_days"]} days')
    print(f'   Process ID: {info["process_id"]}')
    
    # Memory stats
    mem_info = r.info('memory')
    used_mb = mem_info['used_memory'] / 1024 / 1024
    peak_mb = mem_info['used_memory_peak'] / 1024 / 1024
    print(f'\n💾 Memory:')
    print(f'   Used: {used_mb:.2f} MB')
    print(f'   Peak: {peak_mb:.2f} MB')
    print(f'   Fragmentation: {mem_info.get("mem_fragmentation_ratio", 1.0):.2f}')
    
    # Stats
    stats = r.info('stats')
    print(f'\n📈 Stats:')
    print(f'   Total connections: {stats["total_connections_received"]}')
    print(f'   Total commands: {stats["total_commands_processed"]}')
    print(f'   Keyspace hits: {stats.get("keyspace_hits", 0)}')
    print(f'   Keyspace misses: {stats.get("keyspace_misses", 0)}')
    
    # Calculate hit rate
    hits = stats.get('keyspace_hits', 0)
    misses = stats.get('keyspace_misses', 0)
    total = hits + misses
    if total > 0:
        hit_rate = (hits / total) * 100
        print(f'   Hit rate: {hit_rate:.1f}%')
    
    # Clients
    clients = r.info('clients')
    print(f'\n👥 Clients:')
    print(f'   Connected: {clients["connected_clients"]}')
    print(f'   Blocked: {clients.get("blocked_clients", 0)}')
    
    # Keyspace
    keyspace = r.info('keyspace')
    print(f'\n🔑 Keyspace:')
    if keyspace:
        for db, info in keyspace.items():
            print(f'   {db}: {info}')
    else:
        print('   No keys found')

# Run diagnostics
run_diagnostics()


---

## Part 3: SLOWLOG Analysis

Identify slow queries affecting performance:


In [ ]:
def analyze_slowlog():
    """Analyze slow query log"""
    
    print('🐌 Slow Query Analysis')
    print('=' * 60)
    
    # Get slowlog entries
    slowlog = r.slowlog_get(10)  # Last 10 entries
    
    if not slowlog:
        print('✅ No slow queries found!')
        print('   This is good - your queries are fast')
        return
    
    print(f'\n⚠️  Found {len(slowlog)} slow queries:')
    print()
    
    for entry in slowlog:
        # entry is a dict with: id, start_time, duration, command
        duration_ms = entry['duration'] / 1000  # Convert to ms
        command = ' '.join(str(arg) for arg in entry['command'])
        
        print(f'⏱️  Duration: {duration_ms:.2f} ms')
        print(f'   Command: {command[:100]}')
        print()

# First, generate some activity
print('📝 Generating sample queries...')
for i in range(100):
    r.set(f'test:{i}', f'value_{i}')
    r.get(f'test:{i}')

# Analyze slowlog
analyze_slowlog()

# Show slowlog configuration
print('⚙️  Slowlog Configuration:')
threshold = r.config_get('slowlog-log-slower-than')
max_len = r.config_get('slowlog-max-len')
print(f'   Threshold: {threshold} microseconds')
print(f'   Max entries: {max_len}')


---

## Part 4: CLIENT LIST Analysis

Monitor connected clients and identify issues:


In [ ]:
def analyze_clients():
    """Analyze connected clients"""
    
    print('👥 Client Connection Analysis')
    print('=' * 60)
    
    # Get client list
    clients = r.client_list()
    
    if not clients:
        print('ℹ️  No clients connected')
        return
    
    print(f'\n📊 Total clients: {len(clients)}')
    print()
    
    # Analyze clients
    idle_times = []
    commands = []
    
    for client in clients:
        idle = int(client.get('idle', 0))
        idle_times.append(idle)
        
        cmd = client.get('cmd', 'unknown')
        commands.append(cmd)
    
    # Show summary
    if idle_times:
        avg_idle = sum(idle_times) / len(idle_times)
        max_idle = max(idle_times)
        
        print(f'⏱️  Idle Times:')
        print(f'   Average: {avg_idle:.1f} seconds')
        print(f'   Maximum: {max_idle} seconds')
        
        # Warn about idle connections
        long_idle = [c for c in clients if int(c.get('idle', 0)) > 300]
        if long_idle:
            print(f'\n⚠️  {len(long_idle)} clients idle > 5 minutes')
            print('   Consider setting timeout for idle connections')
    
    # Show sample clients
    print(f'\n📋 Sample Clients (showing first 3):')
    for i, client in enumerate(clients[:3]):
        print(f'\n   Client {i+1}:')
        print(f'      Address: {client.get("addr", "unknown")}')
        print(f'      Age: {client.get("age", 0)} seconds')
        print(f'      Idle: {client.get("idle", 0)} seconds')
        print(f'      Last command: {client.get("cmd", "unknown")}')

# Run analysis
analyze_clients()


---

## Part 5: Memory Analysis

Deep dive into memory usage patterns:


In [ ]:
def analyze_memory_by_pattern():
    """Analyze memory usage by key patterns"""
    
    print('💾 Memory Usage by Key Pattern')
    print('=' * 60)
    
    # Create sample data with different patterns
    patterns = {
        'cache:': 50,
        'session:': 30,
        'user:': 20,
    }
    
    for pattern, count in patterns.items():
        for i in range(count):
            r.set(f'{pattern}{i}', 'x' * 100)
    
    # Analyze patterns
    print('\n📊 Key Distribution:')
    print()
    
    total_keys = 0
    for pattern in patterns.keys():
        keys = r.keys(f'{pattern}*')
        count = len(keys)
        total_keys += count
        
        # Sample memory usage
        if keys:
            sample_size = r.memory_usage(keys[0]) if hasattr(r, 'memory_usage') else 100
            estimated_mb = (count * sample_size) / 1024 / 1024
            
            print(f'   {pattern:<12} {count:>5} keys  ~{estimated_mb:.2f} MB')
    
    print(f'\n   Total: {total_keys} keys')
    
    # Get overall memory
    info = r.info('memory')
    used_mb = info['used_memory'] / 1024 / 1024
    print(f'\n💾 Total memory used: {used_mb:.2f} MB')

# Run analysis
analyze_memory_by_pattern()


---

## Part 6: Data Migration Preparation

Prepare for migrating data between Redis instances:


In [ ]:
def prepare_migration_report():
    """Generate pre-migration report"""
    
    print('📋 Pre-Migration Checklist')
    print('=' * 60)
    
    # Count keys
    total_keys = r.dbsize()
    print(f'\n📊 Data Inventory:')
    print(f'   Total keys: {total_keys:,}')
    
    # Sample key types
    if total_keys > 0:
        keys_sample = r.keys('*')[:100]  # Sample first 100
        types = {}
        
        for key in keys_sample:
            key_type = r.type(key)
            types[key_type] = types.get(key_type, 0) + 1
        
        print(f'\n🔑 Key Types (sample of {len(keys_sample)}):')
        for key_type, count in types.items():
            print(f'   {key_type:<10} {count:>5} keys')
    
    # Memory info
    mem_info = r.info('memory')
    used_mb = mem_info['used_memory'] / 1024 / 1024
    peak_mb = mem_info['used_memory_peak'] / 1024 / 1024
    
    print(f'\n💾 Memory Requirements:')
    print(f'   Current usage: {used_mb:.2f} MB')
    print(f'   Peak usage: {peak_mb:.2f} MB')
    print(f'   Recommended target: {peak_mb * 1.2:.2f} MB (with 20% buffer)')
    
    # Estimate migration time
    if total_keys > 0:
        # Rough estimate: 1000 keys/second
        estimated_seconds = total_keys / 1000
        estimated_minutes = estimated_seconds / 60
        
        print(f'\n⏱️  Migration Estimate:')
        print(f'   Estimated time: {estimated_minutes:.1f} minutes')
        print(f'   (at ~1000 keys/second)')
    
    # Checklist
    print(f'\n✅ Pre-Migration Checklist:')
    checklist = [
        'Backup source Redis instance',
        'Provision target instance with sufficient memory',
        'Test connectivity to target instance',
        'Plan maintenance window',
        'Prepare rollback plan',
        'Notify stakeholders',
    ]
    
    for item in checklist:
        print(f'   □ {item}')

# Generate report
prepare_migration_report()


---

## Part 7: Simple Key Migration

Migrate keys between Redis instances (demo with same instance):


In [ ]:
def migrate_keys_demo():
    """Demonstrate key migration pattern"""
    
    print('🚀 Key Migration Demo')
    print('=' * 60)
    
    # Create sample data in "source" namespace
    print('\n📝 Creating source data...')
    source_prefix = 'source:'
    target_prefix = 'target:'
    
    # Clear any existing data
    for key in r.keys(f'{source_prefix}*') + r.keys(f'{target_prefix}*'):
        r.delete(key)
    
    # Create source data
    for i in range(10):
        r.set(f'{source_prefix}key:{i}', f'value_{i}')
        r.setex(f'{source_prefix}temp:{i}', 3600, f'temp_{i}')  # With TTL
    
    source_keys = r.keys(f'{source_prefix}*')
    print(f'   Created {len(source_keys)} source keys')
    
    # Migration function
    def migrate_key(source_key, target_key):
        """Migrate a single key with its TTL"""
        # Get value
        value = r.get(source_key)
        if value is None:
            return False
        
        # Get TTL
        ttl = r.ttl(source_key)
        
        # Set in target
        if ttl > 0:
            r.setex(target_key, ttl, value)
        else:
            r.set(target_key, value)
        
        return True
    
    # Migrate keys
    print(f'\n🔄 Migrating keys...')
    migrated = 0
    
    for source_key in source_keys:
        # Convert source: prefix to target: prefix
        target_key = source_key.replace(source_prefix, target_prefix)
        
        if migrate_key(source_key, target_key):
            migrated += 1
    
    print(f'   Migrated {migrated} keys')
    
    # Verify migration
    target_keys = r.keys(f'{target_prefix}*')
    print(f'\n✅ Verification:')
    print(f'   Source keys: {len(source_keys)}')
    print(f'   Target keys: {len(target_keys)}')
    print(f'   Migration complete: {len(source_keys) == len(target_keys)}')
    
    # Show sample
    if target_keys:
        sample_key = target_keys[0]
        sample_value = r.get(sample_key)
        sample_ttl = r.ttl(sample_key)
        
        print(f'\n📋 Sample migrated key:')
        print(f'   Key: {sample_key}')
        print(f'   Value: {sample_value}')
        print(f'   TTL: {sample_ttl} seconds' if sample_ttl > 0 else '   TTL: No expiration')

# Run migration demo
migrate_keys_demo()


---

## Part 8: Performance Testing

Test performance before and after migration:


In [ ]:
def benchmark_performance():
    """Benchmark Redis performance"""
    
    print('⚡ Performance Benchmark')
    print('=' * 60)
    
    import statistics
    
    operations = 1000
    
    # Test SET performance
    print(f'\n📝 Testing SET ({operations} operations)...')
    set_times = []
    
    for i in range(operations):
        start = time.perf_counter()
        r.set(f'perf:test:{i}', f'value_{i}')
        elapsed = (time.perf_counter() - start) * 1000
        set_times.append(elapsed)
    
    print(f'   Average: {statistics.mean(set_times):.2f} ms')
    print(f'   Median: {statistics.median(set_times):.2f} ms')
    print(f'   P95: {sorted(set_times)[int(len(set_times) * 0.95)]:.2f} ms')
    print(f'   P99: {sorted(set_times)[int(len(set_times) * 0.99)]:.2f} ms')
    
    # Test GET performance
    print(f'\n📖 Testing GET ({operations} operations)...')
    get_times = []
    
    for i in range(operations):
        start = time.perf_counter()
        r.get(f'perf:test:{i}')
        elapsed = (time.perf_counter() - start) * 1000
        get_times.append(elapsed)
    
    print(f'   Average: {statistics.mean(get_times):.2f} ms')
    print(f'   Median: {statistics.median(get_times):.2f} ms')
    print(f'   P95: {sorted(get_times)[int(len(get_times) * 0.95)]:.2f} ms')
    print(f'   P99: {sorted(get_times)[int(len(get_times) * 0.99)]:.2f} ms')
    
    # Cleanup
    for i in range(operations):
        r.delete(f'perf:test:{i}')
    
    print(f'\n✅ Benchmark complete')

# Run benchmark
benchmark_performance()


---

## Part 9: Troubleshooting Common Issues

### Common Problems and Solutions


In [ ]:
def troubleshooting_guide():
    """Display troubleshooting guide"""
    
    issues = [
        {
            'problem': 'High Memory Usage',
            'symptoms': ['Memory close to limit', 'Evictions occurring'],
            'solutions': [
                'Check for keys without TTL',
                'Analyze key patterns with MEMORY USAGE',
                'Consider using Hashes instead of separate keys',
                'Review maxmemory-policy setting',
                'Scale up to larger instance'
            ]
        },
        {
            'problem': 'Slow Query Performance',
            'symptoms': ['High latency', 'Slow response times'],
            'solutions': [
                'Check SLOWLOG for expensive commands',
                'Avoid KEYS command in production',
                'Use pipelining for multiple operations',
                'Check network latency',
                'Consider using connection pooling'
            ]
        },
        {
            'problem': 'Connection Issues',
            'symptoms': ['Timeout errors', 'Connection refused'],
            'solutions': [
                'Verify network connectivity',
                'Check firewall rules',
                'Verify authentication credentials',
                'Check maxclients setting',
                'Review timeout configuration'
            ]
        },
        {
            'problem': 'Memory Fragmentation',
            'symptoms': ['High RSS memory', 'Fragmentation ratio > 1.5'],
            'solutions': [
                'Monitor fragmentation ratio',
                'Consider enabling active defragmentation',
                'Restart Redis during maintenance window',
                'Review key patterns and sizes'
            ]
        }
    ]
    
    print('🔧 Troubleshooting Guide')
    print('=' * 60)
    
    for i, issue in enumerate(issues, 1):
        print(f'\n{i}. {issue["problem"]}')
        print('   ' + '-' * 40)
        
        print('   Symptoms:')
        for symptom in issue['symptoms']:
            print(f'     • {symptom}')
        
        print('   Solutions:')
        for solution in issue['solutions']:
            print(f'     ✓ {solution}')

# Display guide
troubleshooting_guide()


---

## Cleanup


In [ ]:
# Clean up test data
r.flushdb()
print('✅ Redis data cleaned')

# Stop and remove Docker container
!docker stop workshop-redis-module10
!docker rm workshop-redis-module10

print('✅ Docker container removed')
print('✅ Cleanup complete')

---

## 🎯 Key Takeaways

### 🔍 Diagnostic Tools

1. **INFO Command**
   - Server, memory, stats, clients, keyspace
   - Run regularly for monitoring
   - Track trends over time

2. **SLOWLOG**
   - Identify expensive queries
   - Set appropriate threshold
   - Optimize slow commands

3. **CLIENT LIST**
   - Monitor connections
   - Identify idle clients
   - Track client activity

4. **MEMORY USAGE**
   - Analyze key memory
   - Find memory hogs
   - Optimize data structures

### 🚀 Migration Best Practices

1. **Planning**
   - Inventory source data
   - Estimate migration time
   - Plan maintenance window

2. **Execution**
   - Backup before migration
   - Test with sample data
   - Monitor progress
   - Verify data integrity

3. **Tools**
   - DUMP/RESTORE for single keys
   - RIOT for bulk migration
   - Replication for live migration
   - Custom scripts for complex scenarios

4. **Validation**
   - Compare key counts
   - Verify sample data
   - Test performance
   - Check TTLs preserved

### 🛠️ Troubleshooting Checklist

- ✅ Run INFO to get overview
- ✅ Check SLOWLOG for slow queries
- ✅ Monitor memory usage
- ✅ Review client connections
- ✅ Check fragmentation ratio
- ✅ Verify network connectivity
- ✅ Review configuration settings

---

## 🎉 Outstanding Work!

You now have the skills to troubleshoot and migrate Redis successfully!
